In [ ]:
# Mockup of Orgs Table

In [2]:
# Set up the environment with AWS credentials
import boto3
import uuid
session = boto3.Session(profile_name='default', region_name='us-east-1')
ddb = session.resource('dynamodb')
dynamo = session.client('dynamodb')

# Define the table name
table_name = 'campaigns'

# Create the table
table = ddb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'campaign_name',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'org_name',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'campaign_name',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'org_name',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print(table.table_status)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
print(table.table_status)

CREATING
CREATING


# Add Campaign to Table

In [3]:
# Add a org
org1 = table.put_item(
    Item={
        'campaign_id': str(uuid.uuid4().hex),
        'campaign_name': 'holiday campaign',
        'campaign_template': 'credentials_phishing',
        'org_name': 'acme corp',
        'domain': 'acme.com',
        'landing_page': True,
        'landing_page_url': 'https://acme.com',
        'email_campaign': True,
        'email_template': 'email_template1',
        'campaign_start_date': '2020-01-01',
        'campaign_end_date': '2020-01-31',
        'campaign_status': 'active',
        'targets': ['target1', 'target2'],
        'payload': 's3://payloads/credentials_phishing.html',
        'public': 'false'
    }
)

print(org1)

{'ResponseMetadata': {'RequestId': '01HB016IR6TPM6LNSDAV68VRNFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 05 Mar 2023 13:25:57 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '01HB016IR6TPM6LNSDAV68VRNFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


# Get the Campaign Info

In [4]:
query = table.query(
    KeyConditionExpression=boto3.dynamodb.conditions.Key('campaign_name').eq('holiday campaign')
)
print(query['Items'])

[{'email_template': 'email_template1', 'campaign_status': 'active', 'campaign_end_date': '2020-01-31', 'public': 'false', 'landing_page_url': 'https://acme.com', 'campaign_id': '2a3b74b22a3b4d2a81867cf4da27a09a', 'campaign_name': 'holiday campaign', 'payload': 's3://payloads/credentials_phishing.html', 'targets': ['target1', 'target2'], 'campaign_start_date': '2020-01-01', 'email_campaign': True, 'landing_page': True, 'campaign_template': 'credentials_phishing', 'org_name': 'acme corp', 'domain': 'acme.com'}]


In [ ]:
remove = dynamo.delete_table(TableName=table_name)